# (26) Awake -- sv + gr

**Motivation**: Fit SVINET and Group for CA awake data. <br>

In [1]:
# HIDE CODE


import os, sys
from IPython.display import display

# tmp & extras dir
git_dir = os.path.join(os.environ['HOME'], 'Dropbox/git')
extras_dir = os.path.join(git_dir, 'jb-Ca-fMRI/_extras')
fig_base_dir = os.path.join(git_dir, 'jb-Ca-fMRI/figs')
tmp_dir = os.path.join(git_dir, 'jb-Ca-fMRI/tmp')

# GitHub
sys.path.insert(0, os.path.join(git_dir, '_Ca-fMRI'))
from figures.fighelper import *
from analysis.final import *
from utils.render import *

# warnings, tqdm, & style
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tqdm.notebook import tqdm
%matplotlib inline
set_style()

## Mice

In [2]:
mice = Mice(Config(
    nn=512,
    sub_ids=[2, 6, 7, 9, 10],
    ses_ids=[4],
    run_ids=range(1, 10 + 1),
))
bs = Base(mice, mode='ca2')

In [3]:
total = count_sv_fitted(mice.cfg.svinet_dir, 'p15-sample')
remaining = {
    b: np.round(100 * sum(e == 0 for e in d.values()) / len(d), 1)
    for b, d in total.items()
}
remaining

{'(0.0,5.0)': 78.3, '(0.01,0.5)': 78.3, '(0.01,5.0)': 78.3, '(0.5,5.0)': 78.3}

## Fit svinet

In [4]:
props = {
    'mice': mice,
    'num_k': 7,
    'perc': 'p15-sample',
    'mode': 'ca2',
    'task': 'rest',
    'metric': 'pearson',
    'match_metric': 'euclidean',
    'match_using': 'gam',
    'graph_type': 'real',
    'verbose': False,
}
kws = {
    'task': 'rest',
    'desc_ca2': 'awake',
    'runs_only': True,
    'exclude': True,
}

In [ ]:
pbar1 = tqdm([None, 0.01, 0.5, (0.01, 0.5)])
for b in pbar1:
    kws['band_ca2'] = b
    mice.setup_func_data(**kws)
    pbar2 = tqdm(
        mice.get_data_containers('ca2')[0],
        leave=False,
    )
    for key in pbar2:
        msg = f"running net (task-{props['task']}, "
        msg += f"b-{str(b).replace(' ', '')})"
        msg += f": {key}"
        pbar1.set_description(msg)
        _ = SVINET(key=key, **props).fit_svinet()

## Fit group

In [3]:
props = {
    'mice': mice,
    'num_k': 7,
    'perc': 'p15-sample',
    'mode': 'ca2',
    'metric': 'pearson',
    'sv_props': {
        'match_metric': 'euclidean',
        'match_using': 'gam'},
    'graph_type': 'real',
    'dist_metric': 'cosine',
    'ordering': 'infer',
    'verbose': False,
}
kws = {
    'task': 'rest',
    'desc_ca2': 'awake',
    'runs_only': True,
    'exclude': True,
}

In [ ]:
pbar = tqdm([None, 0.01, 0.5, (0.01, 0.5)])
for b in pbar:
    msg = f"running group (task-{kws['task']}, "
    msg += f"b-{str(b).replace(' ', '')}, kk-{props['num_k']})"
    pbar.set_description(msg)
    
    kws['band_ca2'] = b
    mice.setup_func_data(**kws)
    gr = Group(**props)
    if gr.svinets_exist():
        gr.fit_group(force=False)